In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import os
import shutil

In [2]:
# Acces teh Dataset
import os

#list all files in the input directory
input_dir = '/kaggle/input/cassava-leaf-disease-classification'
os.listdir(input_dir)

['train_tfrecords',
 'sample_submission.csv',
 'test_tfrecords',
 'label_num_to_disease_map.json',
 'train_images',
 'train.csv',
 'test_images']

In [3]:
import pandas as pd
import json

# Load the train.csv file
train_df = pd.read_csv(os.path.join(input_dir, 'train.csv'))

# Load the JSON file for mapping labels to diseases
with open(os.path.join(input_dir, 'label_num_to_disease_map.json')) as f:
    label_map = json.load(f)

# Display the first few rows of the training data
print(train_df.head())

# Display the label mapping
print(label_map)


         image_id  label
0  1000015157.jpg      0
1  1000201771.jpg      3
2   100042118.jpg      1
3  1000723321.jpg      1
4  1000812911.jpg      3
{'0': 'Cassava Bacterial Blight (CBB)', '1': 'Cassava Brown Streak Disease (CBSD)', '2': 'Cassava Green Mottle (CGM)', '3': 'Cassava Mosaic Disease (CMD)', '4': 'Healthy'}
